In [3]:
import os
import pandas as pd
import json
import traceback
from langchain.chat_models import ChatOpenAI

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
KEY = os.getenv("OPENAI_API_KEY")


In [4]:
llm =  ChatOpenAI(api_key= KEY, model_name='gpt-3.5-turbo', temperature=0.5)


In [5]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001BBBA864370>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001BBBA98FD00>, temperature=0.5, openai_api_key='sk-proj-sTBra7HSZD0EvvRj49HpT3BlbkFJi2LpQthkzmz48rkvvg5c', openai_proxy='')

In [7]:
from langchain.llms import openai
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain , SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [9]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [17]:
TEMPLATE1= """       
Text : {text}
You are an expert MCQ maker. Given the above text it is your job to\
create a quiz of {number} multiple choice question for {subject} student in {tone} tone. \
Make sure the questions are not repeated and check all the questions to be conforming to the text as well.
Make sure to format you response like RESPONSE_JSON below and use it as guide.\
Ensure to make {number} MCQs.
### RESPONSE_JSON
{response_json}
"""


In [18]:
quiz_generation_prompt= PromptTemplate(input_variables=['text', 'number','subject', 'tone', 'response_json' ],
               template=TEMPLATE1
               )

In [24]:
quiz_chain= LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [20]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [21]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [22]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [25]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [28]:
file_path=r'D:\pythonWorkspace\MCQ_LLM\data.txt'
file_path


'D:\\pythonWorkspace\\MCQ_LLM\\data.txt'

In [29]:
with open(file_path,'r')  as f:
    text= f.readline()
text

'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]\n'

In [30]:
NUMBER=5 
SUBJECT="Machine learning"
TONE="simple"

In [31]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": text,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
       
Text : Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]

You are an expert MCQ maker. Given the above text it is your job tocreate a quiz of 5 multiple choice question for Machine learning student in simple tone. Make sure the questions are not repeated and check all the questions to be conforming to the text as well.
Make sure to format you response like RESPONSE_JSON below and use it as guide.Ensure to make 5 MCQs.
### RESPONSE_JSON
{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "corre

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [32]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:0
Prompt Tokens:0
Completion Tokens:0
Total Cost:0.0


In [36]:
quiz=response.get('quiz')

NameError: name 'response' is not defined

In [38]:
quiz=json.loads(quiz)

NameError: name 'quiz' is not defined

In [ ]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [ ]:
quiz=pd.DataFrame(quiz_table_data)

In [ ]:

quiz.to_csv("machinelearning.csv",index=False)

In [39]:

from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'05_16_2024_23_46_13'